In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:85%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [3]:
llm = ChatOllama(model="llama3.2:1b")
result = llm.invoke("what is the famous food in china??")
result.content

'China has a diverse and rich culinary culture, with many delicious dishes to explore. Some of the most famous foods in China include:\n\n1. Peking Duck (, Pèi Gōng Dòufàn): a classic dish from Beijing, made with roasted duck that\'s crispy on the outside and juicy on the inside.\n2. Xiaolongbao (, "soup dumpling"): a type of steamed bun filled with pork and broth, served with vinegar and chili oil.\n3. Jiaozi (, "dumplings"): boiled or pan-fried dumplings filled with pork, chicken, or vegetables, typically served with dipping sauce.\n4. Chow Mein (, "stir-fry noodles"): a stir-fried noodle dish made with wheat flour noodles, vegetables, and often meat or seafood.\n5. Wontons (, "dumpling wrapper"): boiled or pan-fried dumplings filled with shrimp or pork, typically served in soup or as an appetizer.\n\nHowever, one of the most iconic and beloved Chinese dishes is:\n\n**Beef Noodle Soup (, Chī Jiǎo Mǐ Guǒ)**\n\nA simple yet comforting dish made with thin noodles, braised beef, and vege

In [4]:
food_prompt_template = PromptTemplate(
    template="What is the most famous  food in {country}? just reply with the food name only.",
    input_variables=["country"])
food_prompt = food_prompt_template.invoke({"country":"china"})
print(food_prompt)
llm.invoke(food_prompt).content

text='What is the most famous  food in china? just reply with the food name only.'


'Peking duck.'

In [5]:
output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(food_prompt))

'Peking roast duck'

In [6]:
output_parser.invoke(llm.invoke(food_prompt_template.invoke({"country":"China"})))

'Peking Roast Duck'

In [7]:
recipe_prompt_template = PromptTemplate(
    template="Give me a very short and simple recipe for {food}. Just 3-5 steps and avoid too much detail.",
    input_variables=['food'])
recipe_prompt = recipe_prompt_template.invoke({"food":"Peking duck"})
print(recipe_prompt)
llm.invoke(recipe_prompt).content

text='Give me a very short and simple recipe for Peking duck. Just 3-5 steps and avoid too much detail.'


"Here's a simplified recipe for Peking duck:\n\nIngredients:\n\n* 1 whole duck\n* 2 tbsp soy sauce\n* 2 tbsp sugar\n* 2 tbsp oyster sauce (optional)\n* Scallions, chopped\n\nSteps:\n\n1. Rinse and pat dry the duck.\n2. Mix together soy sauce, sugar, and oyster sauce (if using) in a bowl.\n3. Rub the mixture all over the duck, making sure to get some under the skin as well.\n4. Roast the duck at 400°F (200°C) for about 30 minutes per pound until crispy skin and cooked through.\n\nNote: You can serve this with pancakes or scallions on top."

In [8]:
output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(recipe_prompt))

"Here's a simplified recipe for Peking duck:\n\nIngredients:\n\n* 1 whole duck\n* 2 tbsp soy sauce\n* 2 tbsp Shaoxing wine (or dry sherry)\n* 2 tbsp cornstarch\n* 2 tbsp vegetable oil\n* Scallions, chopped (for garnish)\n\nSteps:\n\n1. Preheat oven to 400°F (200°C).\n2. Marinate the duck in soy sauce, Shaoxing wine, and cornstarch mixture for at least 30 minutes.\n3. Roast the duck in the preheated oven for about 20-25 minutes per pound until crispy skin and cooked meat are reached.\n4. Garnish with chopped scallions.\n\nNote: Traditionally, Peking duck is roasted in a scorching hot wok over an open flame, but this simplified recipe will still give you a delicious result!"

In [9]:
food_chain = food_prompt_template | llm | output_parser
food_chain.invoke({"country":"china"})

'Pork'

In [10]:
recipe_chain = recipe_prompt_template | llm | output_parser
recipe_chain.invoke({"food":"Bulgogi"})

"Here's a simplified recipe for Bulgogi:\n\nIngredients:\n\n* 1 pound beef (thinly sliced)\n* 2 tablespoons soy sauce\n* 2 tablespoons sugar\n* 2 tablespoons garlic, minced\n* 1 tablespoon sesame oil\n* Salt and black pepper to taste\n* Optional: green onions and ginger for garnish\n\nSteps:\n\n1. Marinate the beef in a mixture of soy sauce, sugar, garlic, and sesame oil.\n2. Grill or pan-fry the marinated beef until browned.\n3. Serve with steamed vegetables and rice.\n\nEnjoy!"

In [15]:
final_chain = {"country":RunnablePassthrough()}|\
                {"food":food_chain} | recipe_chain
final_chain.invoke("china")

"Here's a simplified recipe for Peking duck:\n\nIngredients:\n\n* 1 whole duck\n* 2 tablespoons soy sauce\n* 2 tablespoons Shaoxing wine\n* 1 tablespoon cornstarch\n* 2 scallions, chopped\n* Hoisin sauce and plum sauce (for serving)\n\nSteps:\n\n1. Marinate the duck in a mixture of soy sauce, Shaoxing wine, and cornstarch for at least 30 minutes.\n2. Grill or pan-fry the duck until crispy on the outside and cooked through.\n3. Serve with chopped scallions, hoisin sauce, and plum sauce.\n\nNote: You can also use a pre-made Peking duck kit, which typically includes everything you need except for the grill or pan."

In [16]:
type(food_chain), type(recipe_chain)

(langchain_core.runnables.base.RunnableSequence,
 langchain_core.runnables.base.RunnableSequence)